# Dask Bag

Материалы: 
* Макрушин С.В. Лекция 12: Map-Reduce
* https://docs.dask.org/en/latest/bag.html
* Jesse C. Daniel. Data Science with Python and Dask. 

## Задачи для совместного разбора

1. Считайте файл `Dostoevskiy Fedor. Igrok - BooksCafe.Net.txt` и разбейте на предложения. Подсчитайте длину (в кол-ве символов) каждого предложения.

2. Считайте файл `Dostoevskiy Fedor. Igrok - BooksCafe.Net.txt` и разбейте на предложения. Выведите предложения, длина которых не более 10 символов.

3. На основе списка предложений из задачи 1-2 создайте `dask.bag`. Рассчитайте среднюю длину предложений в тексте.

4. На основе файла `addres_book.json` создайте `dask.bag`. Посчитайте количество мобильных и рабочих телефонов в наборе данных

## Лабораторная работа 12

In [ ]:
pip install fsspec

     |████████████████████████████████| 132 kB 5.0 MB/s 


In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
import dask.bag as db
import json
import re
import pandas as pd

1. В файлах архиве `reviews_full.zip` находятся файлы, содержащие информацию об отзывах к рецептам в формате JSON Lines. Отзывы разделены на файлы в зависимости от оценки (например, в файле `reviews_1.json` находятся отзывы с оценкой 1). Считайте файлы из этого архива в виде `dask.bag`. Преобразуйте текстовое содержимое файлов с помощью модуля `json`. Выведите на экран первые 5 элементов полученного `bag`.

In [ ]:
b = db.read_text('/content/reviews_full.zip' ).map(json.loads)
b

dask.bag<loads, npartitions=1>

#WARNING TEST


In [ ]:
# rev= db.read_text('/content/danns/reviews_0.json').take(3)
# result = list(rev)
# result
for i in range(4):
  rev = db.read_text(f'/content/danns/reviews_{i}.json').map(json.loads).take(5)

  result = re.findall(r'\w', f'/content/danns/reviews_{i}.json')
  rating = result[20]
  print (type(rev[0]))
  
  rev[0].update({"rating":rating})
  print (rev[0])

  


In [ ]:
def jss (i):
  rev = db.read_text(f'/content/danns/reviews_{i}.json').map(json.loads).take(5)

  result = re.findall(r'\w', f'/content/danns/reviews_{i}.json')
  rating = result[20]
  print (type(rev[0]))
  
  rev[0].update({"rating":rating})
  print (rev[0])


b.to_textfiles("/content/sample_data/*.json")  
#b.to_textfiles("/content/sample_data/*.json", name_function=jss)  

['/content/sample_data/0.json']

In [ ]:
import dask
import json
import os

k = dask.datasets.make_people()                 # Make records of people
k.map(json.dumps).to_textfiles('/content/test/*.json')

['/content/test/0.json',
 '/content/test/1.json',
 '/content/test/2.json',
 '/content/test/3.json',
 '/content/test/4.json',
 '/content/test/5.json',
 '/content/test/6.json',
 '/content/test/7.json',
 '/content/test/8.json',
 '/content/test/9.json']

In [ ]:
b = db.read_text('/content/test/*.json').map(json.loads)
b

dask.bag<loads, npartitions=10>

In [ ]:
b.filter(lambda record: record['age'] > 30).take(2)

10000

#WARNING END OF TEST

2. Модифицируйте функцию разбора JSON таким образом, чтобы в каждый словарь c информацией об отзыве добавить ключ `rating`. Значение получите на основе названия файла (см. аргумент `include_path`), использовав для этого регулярное выражение.

In [ ]:
def json_rating(files):
  rating = json.loads(files[0])
  comp=re.compile("reviews_(.*?)\.json")
  rating['rating']= int(re.findall(comp, files[1])[0])
  return rating

In [ ]:
output_with_rating = db.read_text(f'/content/drive/MyDrive/Colab Notebooks/lab 12/reviews_*.json', include_path=True).map(json_rating)

3. Посчитайте количество отзывов в исходном датасете.

In [ ]:
output_with_rating.count().compute()

9057540

4. Отфильтруйте `bag`, сохранив только отзывы, оставленные в 2014 и 2015 годах.

In [ ]:
b.filter(lambda record: record['date'] > '2013-31-12' and record['date'] < '2016-01-01').take(5)

/usr/local/lib/python3.7/dist-packages/dask/bag/core.py:2459: UserWarning: Insufficient elements for `take`. 5 elements requested, only 0 elements available. Try passing larger `npartitions` to `take`.
  "larger `npartitions` to `take`.".format(n, len(r))


()

In [ ]:
RESULT = output_with_rating.filter(lambda record: record['date'] > '2013-31-12' and record['date'] < '2016-01-01')

5. Выполните препроцессинг отзывов:
    * привести строки к нижнему регистру
    * обрезать пробельные символы в начале и конце строки
    * удалите все символы, кроме английских букв и пробелов
    
Примените препроцессинг ко всем записям из `bag`, полученного в задании 4.

In [ ]:
def P_O_Y(files):
  pre = re.compile(r'[^a-zA-Z\s]')
  files['review'] = pre.sub('', files['review'].lower().strip())
  return (a)


In [ ]:
RESULT.map(P_O_Y).take(10)

({'date': '2014-10-03',
  'rating': 0,
  'recipe_id': 1300038,
  'review': 'took this to a new years eve party everyone loved it its absolutely perfect the flavor the crunch just delicious',
  'user_id': 229850},
 {'date': '2015-05-08',
  'rating': 0,
  'recipe_id': 133747,
  'review': 'simple and easy way to enjoy a slice of pizza any time  welltoasted bread is the key  really toast it  i put a bit of pizza sauce underneath my cheese for a more pizzalike flavor  i used sourdough bread  medium cheddar cheese  fast  fun  great idea  made for  hits tag game',
  'user_id': 2706705},
 {'date': '2015-06-30',
  'rating': 0,
  'recipe_id': 898468,
  'review': 'delish  i wanted to make this spicy so i used hot enchilada sauce and jalapeno refried beans  i forgot to buy the onions so i doctored up the beans with onion powder and granulated garlic  added the olives under the cheese and baked uncovered for the  minutes  served with pico de gallo sour cream and avocado chunks  fantastic  thanks fo

6. Посчитайте количество отзывов в датасете, полученном в результате решения задачи 5. В случае ошибок прокомментируйте результат и исправьте функцию препроцессинга.

In [ ]:
RESULT.count().compute()

735274

7. Посчитайте, как часто в наборе, полученном в задании 5, встречается та или иная оценка.

In [ ]:
#RESULT.distinct(key=lambda x: x['rating']).compute()
TEMP_RES=RESULT.pluck('rating')
TEMP_RES.frequencies().compute()

[(0, 42472), (1, 9246), (2, 9380), (3, 26532), (4, 119413), (5, 528231)]

8. Найдите среднее значение `rating` в наборе, полученном в задании 5.

In [ ]:
TEMP_RES.mean().compute()

4.388036296673077

9. Используя метод `foldby`, подсчитать максимальную длину отзывов в зависимости от оценки `rating` в наборе, полученном в задании 5.

In [ ]:
def binop(total, min):
  if len(min['review']) > total:
      return len(min['review'])
  return total

data.foldby('rating', binop,0).compute()

[(0, 6972), (1, 5567), (2, 6972), (3, 4954), (4, 8587), (5, 8587)]